# Multiclass cancer diagnosis using tumor gene expression signatures
#### By Bertold Vincze and Antonino Rossi

In this assignment, we tried to replicate a portion of the experiments described in [this](https://www.pnas.org/doi/10.1073/pnas.211566398) paper.

218 snap-frozen tumoral samples, as well as 90 normal samples, were collected from patients with multiple types of cancer. The idea behind these experiments will be to use gene expression signatures to predict the types of cancer present in the samples and to try and offer a generalisation over a SVM model.

## Chapter 1 - The Original Paper

### Background

Cancer classification relies on the subjective interpretation of both clinical and histopathological information with an eye toward placing tumors in currently accepted categories based on the tissue of origin of the tumor. However, clinical information can be incomplete or misleading. In addition, there is a wide spectrum in cancer morphology and many tumors are atypical or lack morphologic features that are useful for differential diagnosis. These difficulties can result in diagnostic confusion, prompting calls for mandatory second opinions in all surgical pathology cases. In the aggregate, these are significant limitations that may hinder patient care, add expense, and confound the results of clinical trials.

Molecular diagnostics offer the promise of precise, objective, and systematic human cancer classification, but these tests are not widely applied because characteristic molecular markers for most solid tumors have yet to be identified. DNA microarray-based tumor gene expression profiles have been used for cancer diagnosis. However, studies have been limited to few cancer types and have spanned multiple technology platforms complicating comparison among different datasets.

### Objective

Develop a multi-class classification model among 14 cancer classes, leveraging support vector machines (SVMs).

### Data

The experiments was conducted (and will be thus reproduced) on snap-frozen human tumor and normal tissue specimens, spanning 14 different tumor classes. Such materials were originally collected from the National Cancer Institute/Cooperative Human Tissue Network, Massachusetts General Hospital Tumor Bank, Dana–Farber Cancer Institute, Brigham and other such institutions in Boston area.

218 cancerous samples, as well as 90 healthy samples, were then subjected to oligonucleotide microarray gene expression analysis.

Oligonucleotide microarray gene expression analysis is a technique used to measure the expression levels of large numbers of genes simultaneously. This technique is used to study the expression of thousands of genes at once, and to identify genes that are differentially expressed in different samples.

![microarray](./imgs/microarray.png)

### Accomplishments

The original paper established an SVM architecture with a 78% accuracy. The goal behind this experimentation is to try out the strategies used in the paper as well as some architectures who weren't present at the time of its writing to explore some further results.

## Chapter 2 - Data Exploration

### File Formats

The data used for this study was provided in form of `.res` files. The RES file format is a tab delimited file format that describes an expression dataset. It is organized as follows. The main difference between RES and GCT file formats is the RES file format contains labels for each gene's absent (A) versus present (P) \[and sometimes marginal (M)] calls as generated by Affymetrix's GeneChip software.

More information about this file format can be found [here](https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#RES:_ExpRESsion_.28with_P_and_A_calls.29_file_format_.28.2A.res.29).

![res files](./imgs/res.png)

At the time of our research, no python package was provided for working with `.res` files, so we had to accomplish some manual labor to read the data in and start our analysis.

### Provided Data

The data describes 218 snap-frozen tumoral samples, as well as 90 normal samples, which were collected from patients with multiple types of cancer. For each sample, a description and an accession were provided, followed by a list of candidate genes, which are known or suspected to bear correlation with carcinogenics.

Each of those genes contains two statistics: the gene expression value (i.e. the amount of mRNA detected in the sample) and the expression class, which categorises with the letters `A`, `M` and `P` the value of the gene expression; specifically, they stand for:

- `A`: Absent
- `M`: Marginal
- `P`: Present

The letter-based classification is based on the specific machinery used to measure the microarrays in the paper (Affymetrix GeneChip), and thus we don't have a direct way to verify the inner workings or the accuracy of these measures.


In [1]:
import polars as pl

df = pl.read_csv("data/GCM_Total.res", separator="\t", truncate_ragged_lines=True)

df.describe()

statistic,Description,Accession,Tumor__Breast_Adeno_09-B_003A,,Tumor__Breast_Adeno_09-B_005A,_duplicated_0,Tumor__Breast_Adeno_92_I_078,_duplicated_1,Tumor__Breast_Adeno_93_I_192,_duplicated_2,Tumor__Breast_Adeno_93_I_250,_duplicated_3,Tumor__Breast_Adeno_94_I_155,_duplicated_4,Tumor__Breast_Adeno_94_I_159,_duplicated_5,Tumor__Breast_Adeno_95_I_029,_duplicated_6,Tumor__Breast_Adeno_9912c068_CC,_duplicated_7,Tumor__Breast_Adeno_mBRT1_(8697),_duplicated_8,Tumor__Breast_Adeno_mBRT2_(9078),_duplicated_9,Tumor__Prostate_Adeno_94_I_052,_duplicated_10,Tumor__Prostate_Adeno_95_I_249,_duplicated_11,Tumor__Prostate_Adeno_95_I_256,_duplicated_12,Tumor__Prostate_Adeno_LocalCaP10T,_duplicated_13,Tumor__Prostate_Adeno_LocalCaP1T,_duplicated_14,Tumor__Prostate_Adeno_P_0025,_duplicated_15,…,Normal__Pancreas_Pan13N,_duplicated_261,Normal__Pancreas_Pan_14N,_duplicated_262,Normal__Pancreas_Pan_43N,_duplicated_263,Normal__Pancreas_Pan_42N,_duplicated_264,Normal__Pancreas_Pan_40N,_duplicated_265,Normal__Pancreas_Pan_41N,_duplicated_266,Normal__Pancreas_OV_1,_duplicated_267,Normal__Ovary_OV_2,_duplicated_268,Normal__Ovary_OV_3,_duplicated_269,Normal__Ovary_OV_4,_duplicated_270,Normal__Whole_Brain_BRAIN_1,_duplicated_271,Normal__Whole_Brain_BRAIN_2,_duplicated_272,Normal__Whole_Brain_BRAIN_3,_duplicated_273,Normal__Whole_Brain_BRAIN_4,_duplicated_274,Normal__Whole_Brain_BRAIN_5,_duplicated_275,Normal__Cerebellum_Brain_Ncer_NCB1,_duplicated_276,Normal__Cerebellum_Brain_Ncer_S-51,_duplicated_277,Normal__Cerebellum_Brain_Ncer_S-125,_duplicated_278,_duplicated_279
str,str,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,…,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,str
"""count""","""16063""","""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",…,16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""",16063.0,"""16063""","""0"""
"""null_count""","""1""","""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",…,1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""",1.0,"""1""","""16064"""
"""mean""",null,null,46.132211,null,17.945527,null,167.069974,null,16.928469,null,82.844923,null,31.268879,null,33.377825,null,81.76754,null,74.395816,null,26.59995,null,23.171388,null,264.75926,null,31.306792,null,190.747432,null,212.180103,null,175.499533,null,486.394073,null,…,48.55494,null,16.999938,null,18.00884,null,22.133288,null,36.949636,null,18.411505,null,248.076387,null,204.467161,null,153.623663,null,633.144649,null,462.351429,null,124.607857,null,389.946262,null,196.246,null,376.868704,null,474.184455,null,394.028388,null,343.556061,null,null
"""std""",null,null,393.452575,null,294.26962,null,611.572681,null,255.75055,null,468.294695,null,233.358432,null,187.341336,null,469.891508,null,443.67566,null,251.664362,null,228.72525,null,962.057611,null,265.719834,null,907.94976,null,789.732744,null,716.349317,null,1395.743932,null,…,382.850505,null,239.489302,null,252.552377,null,242.8251,null,382.390101

## Chapter 3 - Data Transformation


### Labels vs Raw Values

As the aforementioned letter classification can be deemed as unreliable, we decided to convert the `.res` files into the more commonly used `.gct` format; the only difference between the two file formats being, we omit the letters and only work with the raw gene expression value.

No built-in tool was provided to us by any python library vendor, so we opted to build a [`polars`](https://pola.rs/)-backed pipeline for this scope and convert the files ourselves. We did, however, also kept the original versions around to conduct experiments and validations in the future.

### Extracting relevant features

With hundreds of genes and 14 classes of tumors to analyse, the first step towards getting any meaningful results is to reduce the complexity space. To do so, we implemented a feature engineering pipeline, powered by `featurewiz`.

We decided to limit the scope of usable features to 20 for the boundaries of this experiment. More features and more sophisticated selection techniques could be applied were the study to be expanded.

## Chapter 4 - Notable Statistics

### Top notable expressed genes

#### [UBC Gene](https://www.genecards.org/cgi-bin/carddisp.pl?gene=UBC)

This gene represents a ubiquitin gene, ubiquitin C. The encoded protein is a polyubiquitin precursor. Conjugation of ubiquitin monomers or polymers can lead to various effects within a cell, depending on the residues to which ubiquitin is conjugated. Ubiquitination has been associated with protein degradation, **DNA repair**, **cell cycle** regulation, kinase modification, endocytosis, and regulation of other cell signaling pathways.

![ubc](./imgs/UBC.png)

#### [USP11 gene](https://www.genecards.org/cgi-bin/carddisp.pl?gene=USP11)

The USP11 gene is a Protein Coding gene for the protein Ubiquitin Specific Peptidase. This protein is responsible for the processes of protein ubiquitination, which controls many intracellular processes, including cell cycle progression, transcriptional activation, and signal transduction. Specifically, it promotes cell proliferation by deubiquitinating phosphorylated E2F1

![usp11](./imgs/USP11.png)

#### [RPL34 gene](https://www.genecards.org/cgi-bin/carddisp.pl?gene=RPL34)

RPL34 is a Protein Coding gene, specifically for the Ribosomal Protein L34, part of the so called *ribosomial subunit*, which codes for the creation of ribosomes, the organelles that catalyze protein synthesis.
Ribosomes are comprised of two structures, called 40S and 60S, with RPL34 being a component of the 60S group. Overexpression of this gene has been observed in some cancer cells and might be a precursor to carginogenic pathways.

![rpl34](./imgs/RPL34.png)

## Chapter 5 - Training the models

### SVM

Support Vector Machines (SVM) are supervised learning models used for classification and regression tasks. They work by finding the hyperplane that best separates data points into different classes. The "support vectors" are the data points closest to the decision boundary, influencing its placement. SVM aims to maximize the margin between classes, enhancing generalization. It's effective in high-dimensional spaces and with limited data. SVM can handle linear and nonlinear data through kernel functions, mapping data into higher-dimensional spaces. Widely used in various fields like image recognition, text classification, and bioinformatics, SVM is valued for its robustness and versatility in handling complex datasets.

This was the original approach used in the paper and we tried to replicate it as loyally as possible.

![svm](./imgs/svm.png)

### VGG16

VGG16 is a convolutional neural network (CNN) architecture renowned for its depth and simplicity. Developed by the Visual Geometry Group (VGG) at the University of Oxford, it comprises 16 weight layers, including 13 convolutional layers and 3 fully connected layers. VGG16 is characterized by its uniform architecture, using small receptive filters (3x3) and max-pooling layers. Despite its simplicity, VGG16 demonstrates impressive performance in image classification tasks due to its deep stack of layers, enabling hierarchical feature learning. While computationally expensive, VGG16 has served as a benchmark and foundation for many subsequent CNN architectures in computer vision applications.

VGG16 is the first of many convolutional neural network architectures that could deal with complex multi-dimensional inputs with relatively few layers, and as such we have adapted it for tasks outside image processing.

![vgg16](./imgs/VGG16.png)

### VGG19

VGG16 and VGG19 are both convolutional neural network (CNN) architectures developed by the Visual Geometry Group (VGG) at the University of Oxford. The primary difference lies in their depth: VGG16 has 16 weight layers (13 convolutional and 3 fully connected), while VGG19 has 19 layers (16 convolutional and 3 fully connected). Consequently, VGG19 is deeper, potentially capturing more intricate features but requiring more computational resources. While both architectures use small 3x3 convolutional filters and max-pooling layers, the additional layers in VGG19 might offer slight performance improvements in tasks requiring high-level feature representation, albeit with increased complexity and computational cost.

VGG19 was an improvement upon the already existing VGG16 architecture and we decided to compare the results for both to see if the three extra layers had any merit.

![vgg19](./imgs/VGG19.png)

## Conclusions and Results

### Results Rundown

The support vector machine perfomed with a 82% accuracy in training and a 71% accuracy in testing, which is remarkable albeit comes short from the original 78% we had as a target from the original paper.

VGG16, with a number of tweaks, reached 82% accuracy in training and 75% accuracy in testing, which was mostly brought down by the large number of tumor classes which might be present at a time, most lost yield is given by a partial classification rather than a total mismatch.

VGG19 improves it slightly with a 88% accuracy in training and 80% accuracy in testing, out-performing the results of the original paper; this was probably due to the fact that the model's arch

## Acknowledgments

### Authors

All of the work in this research was made possible by the original authors of the paper: Sridhar Ramaswamy, Pablo Tamayo, Ryan Rifkin, Sayan Mukherjee, Chen-Hsiang Yeang, Michael Angelo, Christine Ladd, Michael Reich, Eva Latulippe, Jill P. Mesirov, Tomaso Poggio, William Gerald, Massimo Loda, Eric S. Lander, and Todd R. Golub.

### Technologies

<img src="./imgs/polars.png" alt="polars" width="200"/>
<br>
<img src="./imgs/pytorch.png" alt="pytorch" width="200"/>
<br>
<img src="./imgs/featurewiz.png" alt="featurewiz" width="200"/>